<a href="https://colab.research.google.com/github/ioiein/Advanced-ML/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import numpy as np
import pandas as pd
import scipy
import sklearn
import sklearn.linear_model
import sklearn.preprocessing
import itertools

**Пункт 1**.\
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.


In [4]:
with open('/content/drive/MyDrive/chgk/results.pkl','rb') as f:
  results = pickle.load(f)
with open('/content/drive/MyDrive/chgk/players.pkl','rb') as f:
  players = pickle.load(f)
with open('/content/drive/MyDrive/chgk/tournaments.pkl','rb') as f:
  tournaments = pickle.load(f)

In [5]:
df_players = pd.DataFrame.from_dict(players, orient='index').set_index('id')
df_tournaments = pd.DataFrame.from_dict(tournaments, orient='index').set_index('id')

In [7]:
tournaments_2019 = df_tournaments[df_tournaments['dateStart'].str[0:4] == "2019"]
tournaments_2020 = df_tournaments[df_tournaments['dateStart'].str[0:4] == "2020"]

In [8]:
tournaments_2019.head(5)

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
4772,Синхрон северных стран. Зимний выпуск,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,Балтийский Берег. 3 игра,2019-01-25T19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,Балтийский Берег. 4 игра,2019-03-01T19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,Балтийский Берег. 5 игра,2019-04-05T19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,ОВСЧ. 6 этап,2019-02-15T20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [9]:
tournaments_2020.head(5)

,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
id,,,,,,,,
4628,Семь сорок,2020-12-30T16:00:00+03:00,2020-12-30T16:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",None,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",{'dateRequestsAllowedTo': '2020-12-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"
4957,Синхрон Биркиркары,2020-02-21T00:00:00+03:00,2020-02-27T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5151,Яровой,2020-08-01T14:00:00+03:00,2020-08-05T14:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 22325, 'name': 'Михаил', 'patronymic':...",{'dateRequestsAllowedTo': '2020-07-24T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5414,Синхрон северных стран,2020-01-03T19:00:00+03:00,2020-01-10T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5477,Онлайн: Синхрон Урюбджирова,2020-04-18T19:00:00+03:00,2020-04-30T19:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 91324, 'name': 'Эрдни', 'patronymic': ...",{'dateRequestsAllowedTo': '2020-04-30T23:55:00...,"{'1': 12, '2': 12, '3': 12}"


In [17]:
#составим DataFrame с полями tournament_id, team_id, position, player_id, mask и mask_len(она понадобится для выкидывания турниров с неодинаковыми масками)
tournament_list = []
player_list = []
team_list = []
mask_list = []
pos_list = []

for tournament_id in results.keys():
    if (tournament_id in np.concatenate((tournaments_2019.index, tournaments_2020.index))):
        for team in results[tournament_id]:
            if 'mask' in team:
                mask = team['mask']
                position = team['position']
                team_id = team['team']['id']
                for member in team['teamMembers']:
                    tournament_list.append(tournament_id)
                    mask_list.append(mask)
                    pos_list.append(position)
                    team_list.append(team_id)
                    player_list.append(member['player']['id'])
df_players_in_tournaments = pd.DataFrame({'tournament_id': tournament_list,
                   'team_id': team_list,
                   'position': pos_list,
                   'player_id': player_list,
                   'mask': mask_list
             })
df_players_in_tournaments = df_players_in_tournaments[~df_players_in_tournaments['mask'].isna()]
df_players_in_tournaments['mask_len'] = df_players_in_tournaments['mask'].str.len()

#оставляем только турниры с одинаковыми масками
same_masks = df_players_in_tournaments.groupby('tournament_id')['mask_len'].nunique().eq(1)
same_masks = same_masks[same_masks]
df_players_in_tournaments = df_players_in_tournaments[df_players_in_tournaments['tournament_id'].isin(same_masks.index)]

In [18]:
df_players_in_tournaments

,tournament_id,team_id,position,player_id,mask,mask_len
0,4772,45556,1.0,6212,111111111011111110111111111100010010,36
1,4772,45556,1.0,18332,111111111011111110111111111100010010,36
2,4772,45556,1.0,18036,111111111011111110111111111100010010,36
3,4772,45556,1.0,22799,111111111011111110111111111100010010,36
4,4772,45556,1.0,15456,111111111011111110111111111100010010,36
...,...,...,...,...,...,...
566075,6456,69918,6.0,129706,100101101100100100010110100100010001010,39
566076,6456,69918,6.0,192901,100101101100100100010110100100010001010,39
566077,6456,63129,7.0,165962,101000100110000110001000101000010100001,39
566078,6456,63129,7.0,154624,101000100110000110001000101000010100001,39


In [19]:
questions = []
for l in df_players_in_tournaments['mask_len']:
    questions.extend(np.arange(1, l + 1))
    
player_question = pd.DataFrame({
    'tournament_id': np.repeat(df_players_in_tournaments['tournament_id'], df_players_in_tournaments['mask_len']),
    'team_id': np.repeat(df_players_in_tournaments['team_id'], df_players_in_tournaments['mask_len']),
    'position': np.repeat(df_players_in_tournaments['position'], df_players_in_tournaments['mask_len']),
    'player': np.repeat(df_players_in_tournaments['player_id'], df_players_in_tournaments['mask_len']),
    'quest_num': questions,
    'correct': list(chain.from_iterable(df_players_in_tournaments['mask']))
})
player_question['tournament_question'] = all_q['tournament_id'].astype(str) + '_' + all_q['question_num'].astype(str)

bad_ans = player_question[player_question['correct'].isin(['?', 'X'])]
player_question = player_question[~player_question['tournamnet_question'].isin(bad_ans['tournament_question'])]
player_question.loc[:, 'correct'] = player_question['correct'].astype(int, copy=False)

NameError: ignored